In [1]:
import cudf as pd
import numpy as np

In [2]:
event_data = pd.read_csv('data/event_data.csv')

In [3]:
event_data.drop(['id', 'event_dtm', 'update_dtm'], inplace=True, axis=1)

In [4]:
users_data = pd.read_csv('data/users_data.csv')

In [5]:
users_data = users_data[~users_data['user_id'].isna()]

In [6]:
users_data.drop('delete_dtm', axis=1, inplace=True)

In [7]:
usdrub = 90.41

In [8]:
# Метод, приводящий значения в долларах США к рублевым значениям в столбце payout

def convert_to_rub(row):
    x = row['payout']
    if row['currency'] == 'USD':
        return x * usdrub
    else:
        return x

In [9]:
user_phones = users_data['user_phone'].to_numpy()
user_test = np.zeros_like(user_phones)

for i, val in enumerate(user_phones):
    user_test[i] = str(val).strip('+')

users_data['user_phone'] = user_test

In [10]:
del user_test

In [11]:
user_phones = users_data['user_phone'].unique()

In [12]:
event_data['payout'] = event_data.apply(convert_to_rub, axis=1)
event_data.drop('currency', inplace=True, axis=1)
event_data['payout'].fillna(0.0, inplace=True)

In [13]:
event_phones = event_data['user_phone'].to_numpy()
np_test = np.zeros_like(event_phones)

for i, val in enumerate(event_phones):
    np_test[i] = str(val).strip('+')

event_data['user_phone'] = np_test

del np_test

event_data = event_data[event_data['user_phone'].isin(user_phones)]

In [14]:
event_data['event_status'].fillna('neutral', inplace=True)

In [15]:
event_data['product_type'].unique()

0    mortgage
1        loan
2    deposits
Name: product_type, dtype: object

In [19]:
event_data_deposits = event_data.loc[event_data['product_type'] == 'deposits']

event_data_loans = event_data.loc[event_data['product_type'] == 'loan']

event_data_mortgage = event_data.loc[event_data['product_type'] == 'mortgage']

In [22]:
event_data_deposits.to_csv('data/event_data_separated/event_data_deposits.csv')

event_data_loans.to_csv('data/event_data_separated/event_data_loans.csv')

event_data_mortgage.to_csv('data/event_data_separated/event_data_mortgage.csv')

In [ ]:
del event_data_deposits

del event_data_loans

del event_data_mortgage